In [1]:
from Metrotv import Metrotv #class news Scrapper
from datetime import timedelta, date, datetime
from IPython.display import clear_output

In [2]:
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

In [3]:
obj = Metrotv()
end_date = date(2018, 8, 8)
start_date = date(2018, 1, 22)
init_page = 1
init_offset = 0
list_cat = [['internasional', 'internasional'], ['otomotif', 'otomotif'], ['ekonomi', 'ekonomi'],['bola','bola'],
            ['olahraga','olahraga'],['teknologi','teknologi'],['hiburan','hiburan'],['rona','rona']]
init_cat = list_cat[2:]

In [4]:
for single_date in daterange(start_date, end_date):
    for cat in init_cat:
        clear_output()
        print(single_date.strftime("%Y-%m-%d"))
        print(cat[0])
        obj.getAllBerita([], init_page, init_offset, cat[0], cat[1], single_date.strftime("%Y/%m/%d"))
        init_page = 1
    init_cat = list_cat

2018-01-22
ekonomi
page  1
http://ekonomi.metrotvnews.com/index/2018/01/22/0
http://ekonomi.metrotvnews.com/mikro/VNx3aLyK-menhub-duga-kecelakaan-lrt-karena-kesalahan-manusia
Insert berita  Menhub Duga Kecelakaan LRT karena Kesalahan Manusia
salah2
page  2
http://ekonomi.metrotvnews.com/index/2018/01/22/30
http://ekonomi.metrotvnews.com/makro/Dkq3O5QN-pemerintah-bidik-inflasi-jadi-3-di-2020
Insert berita  Pemerintah Bidik Inflasi jadi 3% di 2020
salah2
http://ekonomi.metrotvnews.com/globals/GKdQxPpN-2018-sektor-jasa-diperkirakan-topang-pertumbuhan-ekonomi-tiongkok
Insert berita  2018, Sektor Jasa Diperkirakan Topang Pertumbuhan Ekonomi Tiongkok
salah2
http://ekonomi.metrotvnews.com/makro/gNQymavN-tren-alokasi-apbn-untuk-infrastruktur-terus-meningkat
Insert berita  Tren Alokasi APBN untuk Infrastruktur Terus Meningkat
salah2
http://ekonomi.metrotvnews.com/bursa/VNx3aDJK-sesi-siang-ihsg-masih-bertahan-di-zona-hijau
Insert berita  Sesi Siang, IHSG Masih Bertahan di Zona Hijau
salah2
http:

AttributeError: 'str' object has no attribute 'append'